In [1]:
import sys

sys.path.insert(0, "../../src")
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import pylab as pl

from WSMBSS import *
from general_utils import *
from visualization_utils import *

import warnings

warnings.filterwarnings("ignore")
notebook_name = "Nonnegative_Antisparse_Copula_Debug"

In [2]:
rho = 0.0
N = 500000
NumberofSources = 5
NumberofMixtures = 10

SNR = 30  # dB

S = generate_correlated_copula_sources(
    rho=rho,
    df=4,
    n_sources=NumberofSources,
    size_sources=N,
    decreasing_correlation=False,
)

A = np.random.standard_normal(size=(NumberofMixtures, NumberofSources))
X = A @ S

X, NoisePart = addWGN(X, SNR, return_noise=True)

SNRinp = 10 * np.log10(
    np.sum(np.mean((X - NoisePart) ** 2, axis=1))
    / np.sum(np.mean(NoisePart**2, axis=1))
)

print("The following is the mixture matrix A")
display_matrix(A)
print("Input SNR is : {}".format(SNRinp))

The following is the mixture matrix A


<IPython.core.display.Math object>

Input SNR is : 30.003923293737166


In [3]:
gamma_start = 0.05
gamma_stop = 1e-3

gammaM_start = np.array([gamma_start, gamma_start])
gammaM_stop = np.array([gamma_stop, gamma_stop])
gammaW_start = np.array([gamma_start, gamma_start])
gammaW_stop = np.array([gamma_stop, gamma_stop])

OUTPUT_COMP_TOL = 1e-6
MAX_OUT_ITERATIONS = 3000
LayerGains = [0.5, 0.5]
LayerGains = [1, 1]
LayerMinimumGains = [0.2, 0.2]
LayerMaximumGains = [1e6, 5001]
WScalings = [0.005, 0.005]
GamScalings = [1, 1]
zeta = 1 * 1e-3
beta = 0.5
muD = [0.01, 0.01]

s_dim = S.shape[0]
x_dim = X.shape[0]
h_dim = s_dim
samples = S.shape[1]
W_HX = np.eye(h_dim, x_dim)
W_YH = np.eye(s_dim, h_dim)

# OPTIONS FOR synaptic_lr_rule: "constant", "divide_by_log_index", "divide_by_index"
synaptic_lr_rule = "divide_by_log_index"
# OPTIONS FOR neural_loop_lr_rule: "constant", "divide_by_loop_index", "divide_by_slow_loop_index"
neural_loop_lr_rule = "divide_by_slow_loop_index"

synaptic_lr_decay_divider = 5000
neural_lr_decay_multiplier = 0.005
hidden_layer_gain = 10

In [4]:
debug_iteration_point = 25000
model = OnlineWSMBSS(
    s_dim=s_dim,
    x_dim=x_dim,
    h_dim=h_dim,
    gammaM_start=gammaM_start,
    gammaM_stop=gammaM_stop,
    gammaW_start=gammaW_start,
    gammaW_stop=gammaW_stop,
    beta=beta,
    zeta=zeta,
    muD=muD,
    WScalings=WScalings,
    GamScalings=GamScalings,
    W_HX=W_HX,
    W_YH=W_YH,
    DScalings=LayerGains,
    LayerMinimumGains=LayerMinimumGains,
    LayerMaximumGains=LayerMaximumGains,
    neural_OUTPUT_COMP_TOL=OUTPUT_COMP_TOL,
    set_ground_truth=True,
    S=S,
    A=A,
)

In [5]:
W_HX = model.W_HX
W_YH = model.W_YH
M_H = model.M_H
M_Y = model.M_Y
D1 = model.D1
D2 = model.D2

np.random.seed(100)
H = np.zeros((h_dim, samples))
Y = np.zeros((s_dim, samples))

# H = np.random.randn(h_dim, samples)  # *0.05
# Y = np.random.randn(s_dim, samples)  # *0.05

i_sample = 287

x_current = X[:, i_sample]  # Take one input
y = Y[:, i_sample]
h = H[:, i_sample]

neural_dynamic_iterations = 750
neural_lr_decay_multiplier = 0.005
neural_lr_start = 0.5
neural_lr_stop = 0.001
neural_OUTPUT_COMP_TOL = 1e-5

In [6]:
h,y, oc = model.run_neural_dynamics_nnantisparse_jit(
                    x_current=x_current,
                    h=h,
                    y=y,
                    M_H=M_H,
                    M_Y=M_Y,
                    W_HX=W_HX,
                    W_YH=W_YH,
                    D1=D1,
                    D2=D2,
                    beta=beta,
                    zeta=zeta,
                    neural_dynamic_iterations=neural_dynamic_iterations,
                    lr_start=neural_lr_start,
                    lr_stop=neural_lr_stop,
                    lr_rule=neural_loop_lr_rule,
                    lr_decay_multiplier=neural_lr_decay_multiplier,
                    hidden_layer_gain=hidden_layer_gain,
                    OUTPUT_COMP_TOL=neural_OUTPUT_COMP_TOL,
)

In [7]:
h, y

(array([-0.01484513, -0.74788565,  0.9415959 , -0.27612382,  0.38055214]),
 array([0.        , 0.        , 0.94157714, 0.        , 0.38054455]))

In [10]:
i_sample = 0
MU_MH = np.max(
    [
        gammaM_start[0]
        / (1 + np.log(2 + (i_sample // synaptic_lr_decay_divider))),
        gammaM_stop[0],
    ]
)
MU_MY = np.max(
    [
        gammaM_start[1]
        / (1 + np.log(2 + (i_sample // synaptic_lr_decay_divider))),
        gammaM_stop[1],
    ]
)
MU_WHX = np.max(
    [
        gammaW_start[0]
        / (1 + np.log(2 + (i_sample // synaptic_lr_decay_divider))),
        gammaW_stop[0],
    ]
)
MU_WYH = np.max(
    [
        gammaW_start[1]
        / (1 + np.log(2 + (i_sample // synaptic_lr_decay_divider))),
        gammaW_stop[1],
    ]
)

M_H = (1 - MU_MH) * M_H + MU_MH * np.outer(h, h)
W_HX = (1 - MU_WHX) * W_HX + MU_WHX * np.outer(h, x_current)

M_Y = (1 - MU_MY) * M_Y + MU_MY * np.outer(y, y)
W_YH = (1 - MU_WYH) * W_YH + MU_WYH * np.outer(y, h)
        
D1derivative = (1 - zeta) * beta * (
            np.sum((np.abs(M_H) ** 2) * D1.T, axis=1)
            - np.sum(np.abs(W_HX) ** 2, axis=1)
        ).reshape(-1, 1) + zeta * (
            1 / D1
        ) 

D2derivative = (1 - zeta) * (1 - beta) * (
            np.sum((np.abs(M_Y) ** 2) * D2.T, axis=1)
            - np.sum(np.abs(W_YH) ** 2, axis=1)
        ).reshape(-1, 1) + zeta * (
            1 / D2
        )

In [26]:
D1

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [15]:
D1secondderivative = []
for i in range(D1derivative.shape[0]):
    D1secondderivative.append((1 - zeta) * beta * (M_H[i,i]**2) * D1[i,0] - zeta * (1 / (D1[i,0]**2)))

In [18]:
np.array(D1secondderivative)

array([0.46944063, 0.48558435, 0.49516024, 0.47161974, 0.47358967])

In [21]:
zeta * (1 / (D1 ** 2))

array([[0.001],
       [0.001],
       [0.001],
       [0.001],
       [0.001]])

In [35]:
D1derivative 

array([[ 0.0009916 ],
       [-0.02031645],
       [-0.00740695],
       [-0.00190571],
       [-0.00037321]])

In [36]:
((1 - zeta) * beta *(np.diag(M_H) ** 2)[:,np.newaxis] * D1 - zeta * (1 / (D1 ** 2)))

array([[0.46944063],
       [0.48558435],
       [0.49516024],
       [0.47161974],
       [0.47358967]])

In [37]:
D1derivative / ((1 - zeta) * beta *(np.diag(M_H) ** 2)[:,np.newaxis] * D1 - zeta * (1 / (D1 ** 2)))

array([[ 0.0021123 ],
       [-0.04183917],
       [-0.01495869],
       [-0.00404078],
       [-0.00078805]])